In [1]:
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

cudnn.benchmark = True


Using device: cuda


In [2]:
base_transform = transforms.ToTensor()

ssl_transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.4, 0.4, 0.4, 0.1),
    transforms.ToTensor()
])

to_pil = transforms.ToPILImage()


In [3]:
dataset = datasets.CIFAR10(
    root="/kaggle/working/data",
    train=True,
    download=True,
    transform=base_transform
)


100%|██████████| 170M/170M [00:04<00:00, 39.1MB/s] 


In [4]:
loader = DataLoader(
    dataset,
    batch_size=256,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    drop_last=True
)

print("Batches per epoch:", len(loader))


Batches per epoch: 195


In [5]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, 3, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=2),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)
        )

    def forward(self, x):
        x = self.net(x)
        return x.view(x.size(0), -1)


In [6]:
class Predictor(nn.Module):
    def __init__(self, dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, dim),
            nn.ReLU(),
            nn.Linear(dim, dim)
        )

    def forward(self, x):
        return self.net(x)


In [7]:
class Projector(nn.Module):
    def __init__(self, in_dim=128, hidden_dim=256, out_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, out_dim)
        )

    def forward(self, x):
        return self.net(x)


In [8]:
def ssl_loss(p, z):
    p = F.normalize(p, dim=1)
    z = F.normalize(z, dim=1)
    return F.mse_loss(p, z)


In [9]:
encoder = Encoder().to(device)
projector = Projector().to(device)
predictor = Predictor().to(device)

target_encoder = Encoder().to(device)
target_encoder.load_state_dict(encoder.state_dict())

for p in target_encoder.parameters():
    p.requires_grad = False

optimizer = torch.optim.Adam(
    list(encoder.parameters()) +
    list(projector.parameters()) +
    list(predictor.parameters()),
    lr=1e-3
)

ema_tau = 0.99


In [10]:
@torch.no_grad()
def feature_variance(z):
    return z.var(dim=0).mean().item()

@torch.no_grad()
def cosine_similarity_mean(z1, z2):
    z1 = F.normalize(z1, dim=1)
    z2 = F.normalize(z2, dim=1)
    return (z1 * z2).sum(dim=1).mean().item()

@torch.no_grad()
def update_target_encoder(online, target, tau):
    for op, tp in zip(online.parameters(), target.parameters()):
        tp.data = tau * tp.data + (1 - tau) * op.data


In [21]:
epochs = 50
os.makedirs("/kaggle/working/checkpoints", exist_ok=True)

for epoch in range(epochs):
    start = time.time()

    total_loss, total_var, total_cos = 0.0, 0.0, 0.0

    for images, _ in loader:
        images = images.to(device)

        view1 = torch.stack([
            ssl_transform(to_pil(img.cpu())) for img in images
        ]).to(device)

        view2 = torch.stack([
            ssl_transform(to_pil(img.cpu())) for img in images
        ]).to(device)

        z1 = encoder(view1)
        z2 = encoder(view2)

        h1 = projector(z1)
        h2 = projector(z2)

        p1 = predictor(h1)
        p2 = predictor(h2)

        with torch.no_grad():
            t1 = projector(target_encoder(view1))
            t2 = projector(target_encoder(view2))

        loss = ssl_loss(p1, t2.detach()) + ssl_loss(p2, t1.detach())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        update_target_encoder(encoder, target_encoder, ema_tau)

        with torch.no_grad():
            total_var += feature_variance(z1)
            total_cos += cosine_similarity_mean(z1, z2)

        total_loss += loss.item()

    print(
        f"Epoch {epoch+1}/{epochs} | "
        f"Loss {total_loss/len(loader):.4f} | "
        f"Var {total_var/len(loader):.4f} | "
        f"Cos {total_cos/len(loader):.4f} | "
        f"Time {(time.time()-start)/60:.2f} min"
    )

    torch.save(
        encoder.state_dict(),
        f"/kaggle/working/checkpoints/encoder_epoch_{epoch+1}.pt"
    )


Epoch 1/50 | Loss 0.0027 | Var 0.0006 | Cos 0.9171 | Time 1.57 min
Epoch 2/50 | Loss 0.0027 | Var 0.0006 | Cos 0.9169 | Time 1.55 min
Epoch 3/50 | Loss 0.0027 | Var 0.0006 | Cos 0.9167 | Time 1.56 min
Epoch 4/50 | Loss 0.0027 | Var 0.0006 | Cos 0.9168 | Time 1.56 min
Epoch 5/50 | Loss 0.0026 | Var 0.0006 | Cos 0.9174 | Time 1.56 min
Epoch 6/50 | Loss 0.0027 | Var 0.0006 | Cos 0.9164 | Time 1.55 min
Epoch 7/50 | Loss 0.0027 | Var 0.0006 | Cos 0.9170 | Time 1.54 min
Epoch 8/50 | Loss 0.0027 | Var 0.0006 | Cos 0.9169 | Time 1.55 min
Epoch 9/50 | Loss 0.0027 | Var 0.0006 | Cos 0.9165 | Time 1.54 min
Epoch 10/50 | Loss 0.0027 | Var 0.0006 | Cos 0.9167 | Time 1.55 min
Epoch 11/50 | Loss 0.0027 | Var 0.0006 | Cos 0.9168 | Time 1.56 min
Epoch 12/50 | Loss 0.0027 | Var 0.0006 | Cos 0.9169 | Time 1.56 min
Epoch 13/50 | Loss 0.0027 | Var 0.0006 | Cos 0.9166 | Time 1.55 min
Epoch 14/50 | Loss 0.0027 | Var 0.0006 | Cos 0.9167 | Time 1.56 min
Epoch 15/50 | Loss 0.0027 | Var 0.0006 | Cos 0.9167 | Tim

In [22]:
torch.save(encoder.state_dict(), "/kaggle/working/encoder_final.pt")


In [23]:
from torchvision import datasets, transforms

# No augmentation for probe
probe_transform = transforms.ToTensor()

train_dataset = datasets.CIFAR10(
    root="/kaggle/working/data",
    train=True,
    download=False,
    transform=probe_transform
)

test_dataset = datasets.CIFAR10(
    root="/kaggle/working/data",
    train=False,
    download=False,
    transform=probe_transform
)

train_loader = DataLoader(
    train_dataset,
    batch_size=256,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=256,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

print("Train batches:", len(train_loader))
print("Test batches:", len(test_loader))


Train batches: 196
Test batches: 40


In [24]:
# Load final SSL weights
encoder = Encoder().to(device)
encoder.load_state_dict(torch.load("/kaggle/working/encoder_final.pt"))
encoder.eval()

# Freeze encoder
for p in encoder.parameters():
    p.requires_grad = False

print("Encoder loaded and frozen.")


Encoder loaded and frozen.


In [25]:
class LinearProbe(nn.Module):
    def __init__(self, in_dim=128, num_classes=10):
        super().__init__()
        self.fc = nn.Linear(in_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

probe = LinearProbe().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(probe.parameters(), lr=1e-3)


In [26]:
probe_epochs = 50

for epoch in range(probe_epochs):
    probe.train()
    total_loss = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            features = encoder(images)

        outputs = probe(features)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Probe Epoch {epoch+1}/{probe_epochs} | Loss {total_loss/len(train_loader):.4f}")


Probe Epoch 1/50 | Loss 2.2475
Probe Epoch 2/50 | Loss 2.1586
Probe Epoch 3/50 | Loss 2.0935
Probe Epoch 4/50 | Loss 2.0439
Probe Epoch 5/50 | Loss 2.0043
Probe Epoch 6/50 | Loss 1.9719
Probe Epoch 7/50 | Loss 1.9443
Probe Epoch 8/50 | Loss 1.9210
Probe Epoch 9/50 | Loss 1.8997
Probe Epoch 10/50 | Loss 1.8813
Probe Epoch 11/50 | Loss 1.8642
Probe Epoch 12/50 | Loss 1.8488
Probe Epoch 13/50 | Loss 1.8344
Probe Epoch 14/50 | Loss 1.8212
Probe Epoch 15/50 | Loss 1.8094
Probe Epoch 16/50 | Loss 1.7976
Probe Epoch 17/50 | Loss 1.7869
Probe Epoch 18/50 | Loss 1.7770
Probe Epoch 19/50 | Loss 1.7678
Probe Epoch 20/50 | Loss 1.7589
Probe Epoch 21/50 | Loss 1.7502
Probe Epoch 22/50 | Loss 1.7418
Probe Epoch 23/50 | Loss 1.7348
Probe Epoch 24/50 | Loss 1.7273
Probe Epoch 25/50 | Loss 1.7202
Probe Epoch 26/50 | Loss 1.7139
Probe Epoch 27/50 | Loss 1.7077
Probe Epoch 28/50 | Loss 1.7018
Probe Epoch 29/50 | Loss 1.6961
Probe Epoch 30/50 | Loss 1.6910
Probe Epoch 31/50 | Loss 1.6854
Probe Epoch 32/50

In [27]:
probe.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        features = encoder(images)
        outputs = probe(features)

        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"\n🔹 Linear Probe Test Accuracy: {accuracy:.2f}%")



🔹 Linear Probe Test Accuracy: 43.10%
